In [ ]:
import torch 
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np

from API_utils.dataset_FEGS import API_FEGS_Class
from API_utils.dataset_api import API_Class

In [ ]:
DATASET_PATH = "C:/Users/asus/Desktop/API/dataset/one_to_one.xls"
DATASET_MAT_PATH = "C:/Users/asus/Desktop/API/dataset/one_to_one.mat"
TEST_DATASET_PATH = "C:/Users/asus/Desktop/API/dataset/test.xlsx"
TEST_DATASET_MAT_PATH = "C:/Users/asus/Desktop/API/dataset/test.mat"
SAVE_MODEL_PATH = "C:/Users/asus/Desktop/API/Model/"
CSV_PATH = "C:/Users/asus/Desktop/API/dataset/Dataset.csv"
DEVICE= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCH = 100
lr=0.0001

In [ ]:
print(DEVICE)

In [ ]:
# train_data = API_FEGS_Class(DATASET_PATH,DATASET_MAT_PATH,'abc')
# test_data = API_FEGS_Class(TEST_DATASET_PATH,TEST_DATASET_MAT_PATH,'test')
train_data = API_Class(CSV_PATH)
test_data = API_Class(CSV_PATH)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, lrate = 0.001, loss_fn = nn.CrossEntropyLoss()):
        super(NeuralNet, self).__init__()


        self.conv1 = nn.Sequential(nn.Conv1d(1,8,kernel_size=1),
                                   nn.BatchNorm1d(8),
                                   nn.LeakyReLU()
        )
        self.conv2 = nn.Sequential(nn.Conv1d(8,16,kernel_size=1),
                                   nn.BatchNorm1d(16),
                                   nn.LeakyReLU()
        )
        self.conv3 = nn.Sequential(nn.Conv1d(1,8,kernel_size=3,stride=2),
                                   nn.BatchNorm1d(8),
                                   nn.LeakyReLU()
        )
        self.conv4 = nn.Sequential(nn.Conv1d(8,16,kernel_size=3,stride=2),
                                   nn.BatchNorm1d(16),
                                   nn.LeakyReLU()
        )

        self.conv5 = nn.Sequential(nn.Conv1d(16,8,kernel_size=3,stride=2),
                                   nn.BatchNorm1d(8),
                                   nn.MaxPool1d(kernel_size=2),
                                   nn.LeakyReLU()
        )
        self.conv6 = nn.Sequential(nn.Conv1d(8,1,kernel_size=3,stride=2),
                                   nn.BatchNorm1d(1),
                                   nn.MaxPool1d(kernel_size=2),
                                   nn.LeakyReLU()
        )

        self.conv7 = nn.Linear(28,7)
        self.conv8 = nn.Linear(7,2)
        self.conv9 = nn.Linear(2,1)

        self.loss_fn = loss_fn
        self.lrate=lrate        
        self.optimizer=optim.SGD(self.parameters(),self.lrate,momentum=0.9)
    def forward(self, rna, protein):
        """Performs a forward pass through your neural net (evaluates f(x)).

        @param x: an (N, in_size) Tensor
        @return y: an (N, out_size) Tensor of output from the network
        """
        '''
        extractor for protein 
        '''
        
        rna = self.conv1(rna)
        rna = self.conv2(rna)
        # print(rna.shape)
        protein = self.conv3(protein)
        
        protein = self.conv4(protein)
        # print(protein.shape)
        x = torch.cat((rna,protein),dim=2)
        x = self.conv5(x)
        x = self.conv6(x)
        # print(x.shape)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = torch.squeeze(x,dim=0)
        return x


In [ ]:
class MLP(nn.Module):
    def __init__(self, lrate = 0.001, loss_fn = nn.CrossEntropyLoss()):
        super(MLP, self).__init__()


        self.ln1 = torch.nn.Linear(640,160)
        self.ln2 = torch.nn.Linear(160,40)
        self.ln3 = torch.nn.Linear(40,10)
        self.ln4 = torch.nn.Linear(10,2)
        self.ln5 = torch.nn.Linear(2,1)

        
        
        self.loss_fn = loss_fn
        self.lrate=lrate        
        self.optimizer=optim.SGD(self.parameters(),self.lrate,momentum=0.9)
    def forward(self, x):
        """Performs a forward pass through your neural net (evaluates f(x)).

        @param x: an (N, in_size) Tensor
        @return y: an (N, out_size) Tensor of output from the network
        """
        '''
        extractor for protein 
        '''
        x = self.ln1(x)
        x = self.ln2(x)
        x = self.ln3(x)
        x = self.ln4(x)
        x = self.ln5(x)
        


        x = torch.squeeze(x,dim=0)
        return x

In [ ]:
train_data_loader = DataLoader(train_data,batch_size=1,shuffle=True)
test_data_loader = DataLoader(test_data,batch_size=1,shuffle=True)
Net = MLP().to(DEVICE)
optimizer = optim.AdamW(params=Net.parameters(),lr=lr)
loss_fn = nn.BCEWithLogitsLoss()


In [ ]:
def evaluate(model_path,test_data_loader,device=DEVICE):
    count=0
    acc = 0
    model = torch.load(model_path)
    for idx, data in enumerate(test_data_loader):
        api_input, api_label = data
        count+=1
        
        api_input = api_input.to(dtype=torch.float32).to(DEVICE)
        # rna_input = api_input[:,0:256+64]
        # print(rna_input)
        # protein_input = api_input[:,256+64:]
        # rna_input = torch.unsqueeze(rna_input,dim=0)
        # protein_input = torch.unsqueeze(protein_input,dim=0)
        #api_label = torch.unsqueeze(api_label,dim=0)
        # rna_input = rna_input.to(dtype=torch.float32).to(DEVICE)
        # protein_input = protein_input.to(dtype=torch.float32).to(DEVICE)
        output = model(api_input).to(DEVICE)
        # print(torch.round(torch.sigmoid(output)))
        if(torch.round(torch.sigmoid(output))==api_label[0]):
            acc+=1

    
    print("Accuracy",acc/count)

In [ ]:
for epoch in range(EPOCH):
    for data in tqdm(train_data_loader):
        api_input, api_label = data
        # print(api_input.shape)
        # print(api_input)
        # break
        api_input = api_input.to(DEVICE).to(dtype=torch.float32)
        api_label = api_label.to(DEVICE).to(dtype=torch.float32)
        # rna_input = api_input[:,0:64+256]
        # print(api_input.shape)
        # protein_input = api_input[:,64+256:]
        # rna_input = torch.unsqueeze(rna_input,dim=0)
        # protein_input = torch.unsqueeze(protein_input,dim=0)

        # rna_input = rna_input.to(dtype=torch.float32)
        # protein_input = protein_input.to(dtype=torch.float32)
        output = Net(api_input)
        # print(output)
        optimizer.zero_grad()    
        Loss = loss_fn(output, api_label)  
        Loss.backward()  
        optimizer.step()
    
    print("Loss",Loss.item())
    save_path = SAVE_MODEL_PATH+f'Epoch={epoch}_lr={lr}.pth'
    torch.save(Net, save_path)
    evaluate(SAVE_MODEL_PATH+f'Epoch={epoch}_lr={lr}.pth',test_data_loader)
    # break
    
torch.save(Net, SAVE_MODEL_PATH+f'Epoch={epoch}_lr={lr}_final.pth')
print('CSX')

In [ ]:
model=torch.load(SAVE_MODEL_PATH+f'Epoch={epoch}_lr={lr}_final.pth')

In [ ]:

acc=0
count=0
for idx, data in enumerate(test_data_loader):
    api_input, api_label = data
    count+=1
    rna_input = api_input[:,0:64]
    protein_input = api_input[:,64:909]
    rna_input = torch.unsqueeze(rna_input,dim=0)
    protein_input = torch.unsqueeze(protein_input,dim=0)
    #api_label = torch.unsqueeze(api_label,dim=0)
    rna_input = rna_input.to(dtype=torch.float32)
    protein_input = protein_input.to(dtype=torch.float32)
    output = model(rna_input,protein_input)
    if(torch.argmax(output)==api_label[0]):
        acc+=1

    
print(acc/count)

